# MOM-SIS 0.25$^\circ$ Diagnostics

This notebook calculates and retains key diagnostics from our mom01v5 simulations. The following experiments are included:

| Experiment Name | Description |
|-----------------|-----------------|
|mom025_nyf | Original simulation, rerun from WOA13 initial conditions.|
|mom025_nyf_salt | New salt restoring file|

**Last updated May 20 2017.**

### Experiments

In [1]:
DataDir = '/g/data3/hh5/tmp/cosima/mom025'
expts = ['mom025_nyf']